In [ ]:
%load_ext autoreload
%autoreload 2


import scqubits as scq

from qiskit_metal.analyses.quantization.lumped_capacitive import load_q3d_capacitance_matrix
from qiskit_metal.analyses.quantization.lom_core_analysis import CompositeSystem, Cell, Subsystem
                        
from scipy.constants import speed_of_light as c_light

import matplotlib.pyplot as plt
%matplotlib inline

# Example: two transmons coupled by a direct coupler

### 1. load transmon cell Q3d simulation results
loading the Maxwell capacitance matrices for the design as shown in the screenshot below

<img src="two_transmons_direct_coupler.png" width=600 height=600 />

where we have two transmons (alice and bob) coupled to each other through a direct coupler. Each transmon is also coupled to its own readout reasonator.

For a simple introduction on Maxwell capacitance matrix, check out the following resources:
https://www.fastfieldsolvers.com/Papers/The_Maxwell_Capacitance_Matrix_WP110301_R02.pdf

In [ ]:
# loading alice's simulation results
path1 = './Q1_TwoTransmon_CapMatrix.txt'
ta_mat, _, _, _ = load_q3d_capacitance_matrix(path1)

In [ ]:
# loading bob's simulation results
path2 = './Q2_TwoTransmon_CapMatrix.txt'
tb_mat, _, _, _ = load_q3d_capacitance_matrix(path2)

### 2. Create LOM cells from capacitance matrices
#### Setting cell objects corresponding to the capacitance simulation results 
`coupler_connector_pad_Q1` and `coupler_connector_pad_Q2` refer to the same node corresponding to the direct coupler between the qubits but are different names in the capacitance matrix results file. In order to merge the two capactiance matrices in the LOM analysis, we need to rename them to be the same name. Also renaming `readout_connector_pad_Q1` and `readout_connector_pad_Q2` for clarity's sake. 

The following three parameters, `ind_dict`, `jj_dict`, `cj_dict`, all have the same structure. Each is a dictionary where the keys are tuples, giving the nodes that a junction is in between, and the values specifying the relevant values associated with the junction. `ind_dict` lets youu specify the junction inductance in nH; `jj_dict` specifies the Josephson junction name (you can give the junction any name you wish; just need to be consistent with the name); `cj_dict` specifies the junction capacitance in fF.

In [ ]:
# cell 1: transmon Alice cell

opt1 = dict(
    node_rename = {'coupler_connector_pad_Q1': 'coupling', 'readout_connector_pad_Q1': 'readout_alice'}, 
    cap_mat = ta_mat,
    ind_dict = {('pad_top_Q1', 'pad_bot_Q1'):10},  # junction inductance in nH
    jj_dict = {('pad_top_Q1', 'pad_bot_Q1'):'j1'},
    cj_dict = {('pad_top_Q1', 'pad_bot_Q1'):2}, # junction capacitance in fF

)
cell_1 = Cell(opt1)


# cell 2: transmon Bob cell
opt2 = dict(
    node_rename = {'coupler_connector_pad_Q2': 'coupling', 'readout_connector_pad_Q2': 'readout_bob'},
    cap_mat = tb_mat,
    ind_dict = {('pad_top_Q2', 'pad_bot_Q2'): 12}, # junction inductance in nH
    jj_dict = {('pad_top_Q2', 'pad_bot_Q2'):'j2'},
    cj_dict = {('pad_top_Q2', 'pad_bot_Q2'):2}, # junction capacitance in fF

)
cell_2 = Cell(opt2)

### 3. Create subsystems
#### Creating the four subsystems, corresponding to the 2 qubits and the 2 readout resonators

`Subsystem` takes three required arguments. The four currently supported system types are `TRANSMON`, `FLUXONIUM`, `TL_RESONATOR` (transmission line resonator) and `LUMPED_RESONATOR`. `nodes` lets you specify which node the subsystem should be mapped to in the cells. They should be consistent with the node names you have given previously. `q_opts` lets specify any optional parameters you want to give. For example, for qubits, you can provide `scqubits` parameters such as `ncut`, `truncated_dim` here.

In [ ]:
# subsystem 1: transmon Alice
transmon_alice = Subsystem(name='transmon_alice', sys_type='TRANSMON', nodes=['j1'])


# subsystem 2: transmon Bob
transmon_bob = Subsystem(name='transmon_bob', sys_type='TRANSMON', nodes=['j2'])


# subsystem 3: Alice readout resonator
q_opts = dict(
    f_res = 8, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
res_alice = Subsystem(name='readout_alice', sys_type='TL_RESONATOR', nodes=['readout_alice'], q_opts=q_opts)


# subsystem 4: Bob readout resonator
q_opts = dict(
    f_res = 7.6, # resonator dressed frequency in GHz
    Z0 = 50, # characteristic impedance in Ohm
    vp = 0.404314 * c_light # phase velocity 
)
res_bob = Subsystem(name='readout_bob', sys_type='TL_RESONATOR', nodes=['readout_bob'], q_opts=q_opts)

### 4. Creat the composite system from the cells and the subsystems

The LOM analysis will automatically remove all non-dynamic nodes. These are nodes that are either exclusively connected to only capacitors or only inductors and are not true degrees of freedom (please check out https://arxiv.org/pdf/2103.10344.pdf or https://cpb-us-w2.wpmucdn.com/campuspress.yale.edu/dist/2/3627/files/2020/10/Vool_Intro_quantum_electromagnetic_circuits.pdf for more information on this). Since we didn't (and didn't have to) simulate the readout resonators, the two nodes, `readout_alice` and `readout_bob`, connected only to other nodes capacitively as specified by the Maxwell capacitance matrices, would be eliminated. But they are actually dynmaic nodes, connected to the inductors (not simulated) of the respective transmission lines and correspond to subsystesm that we want to include in the Hamiltonian of the composite system, hence we liste them as nodes to force keep with the parameter `nodes_force_keep`.

In [ ]:
composite_sys = CompositeSystem(
    subsystems=[transmon_alice, transmon_bob, res_alice, res_bob], 
    cells=[cell_1, cell_2], 
    grd_node='ground_main_plane',
    nodes_force_keep=['readout_alice', 'readout_bob']
)

The `circuitGraph` object encapsulates the lumped model circuit analysis (i.e., LOM analysis) and contain the intermediate as well as final L and C matrices, their inverses needed to construct the Hamiltonian of the composite system. For more details on the meaning and calculation of these matrices, check out https://arxiv.org/pdf/2103.10344.pdf

Just to note that you can use the analysis without needing to know any detail about this object.

In [ ]:
cg = composite_sys.circuitGraph()
print(cg)

### 5. Generate the hilberspace from the composite system, leveraging the scqubits package

In [ ]:
hilbertspace = composite_sys.create_hilbertspace()
print(hilbertspace)

`add_interaction()` adds the interaction terms between the subsystems. Currently, capacitive coupling is supported (which is extracted by from off-diagonal elements in the C matrices, see *eqn 12, 13* in https://arxiv.org/pdf/2103.10344.pdf ) and contribute to the interaction.

In [ ]:
hilbertspace = composite_sys.add_interaction()
hilbertspace.hamiltonian()

### 6. Print the results

print the calculated Hamiltonian parameters from diagnolized composite system Hamiltonian

The diagonal elements of the $\chi$ matrix are the anharmonicities of the respective subsystesm and the off-diagonal the dispersive shifts between them 

In [ ]:
hamiltonian_results = composite_sys.hamiltonian_results(hilbertspace, evals_count=30)

In [ ]:
hamiltonian_results['chi_in_MHz'].to_dataframe()

The $\chi$'s between the subsystems are based on the coupling strengths, $\it{g}$'s between them (which are computed using the coupling capacitance (currently capacitive coupling is supported) and zero point flucuations of the subystem's charge operator at the coupling location)

In [ ]:
composite_sys.compute_gs()

In [ ]:
composite_sys.compute_gs().to_dataframe()

In [ ]:
transmon_alice.h_params

In [ ]:
transmon_bob.h_params